## Install dependencies

In [ ]:
!pip install requests objective-sdk

In [ ]:
import json
import time

import requests
import objective

## Load e-commerce dataset

This is a dataset of 10,000 products from H&M. Source: https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations

In [ ]:
data = json.loads(requests.get("https://d11p8vtjlacpl4.cloudfront.net/demos/ecommerce/hm-10k.json").text)
assert len(data) == 10000
data[0]

## Initiate the Objective API client with your API key

Join the waitlist to get an API key - https://www.objective.inc/waitlist

In [ ]:
from objective import Client

client = Client(api_key="YOUR_API_KEY")

## Upsert objects

Upsert objects using the Objects API https://www.objective.inc/docs/ingestion/API/put-object

In [ ]:
client.object_store.upsert_objects([{"id": obj.get("id"), "object": obj}for obj in data])

## Create multimodal index

Create a multimodal index to search the data. This will index both the text and the images for searching. Docs - https://www.objective.inc/docs/index/api/create-index

In [ ]:
index = client.indexes.create_index(
    index_type="multimodal", fields={
      "searchable": ["detail_desc", "garment_group_name", "section_name", "index_group_name", "index_name", "department_name", "perceived_colour_value_name", "colour_group_name", "graphical_appearance_name", "product_group_name", "product_type_name", "prod_name",],
      "crawlable": ["image_url"],
      "filterable": ["price", "rating", "colour_group_name", "index_name"]
    }
)

## Watch objects get indexed in real time

Indexing occurs in real time, objects are typically available for search within a few seconds of pushing updates.

In [ ]:
status = {}
while status.get("UPLOADED", -1) != 0:
  status = index.status()
  print("\r" + str(status), end="")
  time.sleep(1)

## Search semantically

Run your first search! 🎉 Semantically search the contents of the text and images, effortlessly finding things like "red dresses with white flowers".

In [ ]:
index.search(query="red dress with white flowers", object_fields="title, image_url")

### Go to prod & learn more!

That's it! You've just built your first AI search application in under 5 minutes. Best part is, you can go straight to production with what you've just built.

Learn more:
* Docs - https://www.objective.inc/docs/
* Example apps - https://github.com/objective-inc/examples
* Get help from the founders - lance@objective.inc
* Book a call to learn more about how we can help with your use case - https://www.objective.inc/contact

## Reset

In [ ]:
client.object_store.delete_objects(client.object_store.list_all_objects())

In [ ]:
for idx in client.indexes.list_indexes():
  idx.delete()